In [27]:
from utils import get_data
import numpy as np
import pandas as pd
from datetime import datetime

In [28]:
data = get_data('../data')

In [43]:
def encodeColumn(df, column, providedList=None):
    if(providedList == None):
        providedList = list(df[column].unique())
    notInListValue = len(providedList)
    return [providedList.index(x) if x in providedList else notInListValue for x in df[column]]

def convertDate(date, dateFormat='%m/%d/%Y'):
    if(isinstance(date, str)):
        date = (datetime.strptime(date, dateFormat)).toordinal()
    return date

In [44]:
data['PROPTYPE_CAT'] = encodeColumn(data, 'PROPTYPE', ['SF', 'CC', 'MF'])
print(data[['PROPTYPE','PROPTYPE_CAT']].head(20))

data['STYLE_CAT'] = encodeColumn(data, 'STYLE')
print(data[['STYLE','STYLE_CAT']].head(20))

data['ZIP_CAT'] = encodeColumn(data, 'ZIP')
print(data[['ZIP','ZIP_CAT']].head(20))

data['LISTDATE_CAT'] = [convertDate(x) for x in data['LISTDATE']]
data['SOLDDATE_CAT'] = [convertDate(x) for x in data['SOLDDATE']]
print(data[['LISTDATE', 'LISTDATE_CAT']].head(20))

print(data[data['STATUS'] == 'SLD'][['SOLDDATE', 'SOLDDATE_CAT']].head())

   PROPTYPE  PROPTYPE_CAT
0        SF             0
1        SF             0
2        SF             0
3        SF             0
4        SF             0
5        SF             0
6        SF             0
7        SF             0
8        SF             0
9        SF             0
10       SF             0
11       SF             0
12       SF             0
13       SF             0
14       SF             0
15       SF             0
16       SF             0
17       SF             0
18       SF             0
19       SF             0
                            STYLE  STYLE_CAT
0             Other (See Remarks)          0
1                            Cape          1
2                        Bungalow          2
3                        Bungalow          2
4                        Colonial          3
5                        Colonial          3
6                        Colonial          3
7                        Colonial          3
8                        Colonial          3
9   

# Define Criteria

Find home that have been sold, and remove condos since they the units may have the same address, and trying to match on address. Note this means we will **not** be working with any condo data in this project, since it is hard to identify flipped condos.

We then match to find houses with the same street address.

In [45]:
soldHomes = data[data['STATUS'] == 'SLD']
soldHomes = soldHomes[soldHomes['PROPTYPE'] != 'CC']

In [46]:
soldMLS = soldHomes['MLSNUM'].unique()

In [47]:
# find home with same address
def getHome(data, mls):
    home = soldHomes[soldHomes['MLSNUM'] == mls]
    homes = soldHomes[soldHomes['ZIP'] == home['ZIP'].iloc[0]]
    homes = homes[homes['HOUSENUM1'] == home['HOUSENUM1'].iloc[0]]
    homes = homes[homes['STREETNAME'] == home['STREETNAME'].iloc[0]]
    return homes

# Find Potential Flips

Go through all mls listing to find matching homes.

Remaining homes (ones that are not flips) will end up on the control/non-flippable list.

In [48]:
test_homes = []
control_homes = []

for mls in soldMLS:
    homes = getHome(soldHomes, mls)
    if(homes.shape[0] == 2):
        test_homes.append(homes)
    else:
        control_homes.append(homes)
        

In [49]:
flippedHomes = pd.concat(test_homes)
print(flippedHomes.shape)

controlHomes = pd.concat(control_homes)
print(controlHomes.shape)

(9770, 44)
(126082, 44)


In [50]:
# remove duplicates and sort by mls
flippedHomes = flippedHomes.drop_duplicates(subset='MLSNUM', keep="first")
flippedHomes = flippedHomes.sort_values(by='MLSNUM')

controlHomes = controlHomes.drop_duplicates(subset='MLSNUM', keep="first")

# Build Test and Control Data Sets

Create the control data set from our array of control homes. Then, put all the pre-flipped homes in as "flippable", and the post-flipped homes back int othe control pile (as they are no longer flippable).

In [51]:
# build flip set
flip_data = [];
post_flips = [];
for mls in flippedHomes['MLSNUM']:
    homes = getHome(flippedHomes, mls)
    # check if home has 2 mls rows
    if(homes.shape[0] >= 2):
        pre = homes.iloc[0]
        post = homes.iloc[1]
        # check if it's not the same mls number
        if pre['MLSNUM'] == post['MLSNUM']:
            continue
        # find pre and post flip values
        if pd.to_datetime(pre['SOLDDATE']) > pd.to_datetime(post['SOLDDATE']):
            # post < pre: swap them
            swap = pre
            pre = post
            post = swap
        flip_data.append({
            'FLIPPABLE': "1",
            'MLSNUM': pre['MLSNUM'],
            'SOLDPRICE': pre['SOLDPRICE'],
            'DOM': pre['DOM'], #days on market
            'BEDS': pre['BEDS'],
            'BATHS': pre['BATHS'],
            'SQFT': pre['SQFT'],
            'AGE': pre['AGE'],
            'GARAGE': pre['GARAGE'],
            'PROPTYPE_CAT': pre['PROPTYPE_CAT'],
            'STYLE_CAT': pre['STYLE_CAT'],
            'ZIP_CAT': pre['ZIP_CAT'],
            'SOLDDATE_CAT': pre['SOLDDATE_CAT'],
        })
        post_flips.append({ # we want to add our post-flip data to the control, since these are not flippable anymore
            'FLIPPABLE': "0",
            'MLSNUM': post['MLSNUM'],
            'SOLDPRICE': post['SOLDPRICE'],
            'DOM': post['DOM'],
            'BEDS': post['BEDS'],
            'BATHS': post['BATHS'],
            'SQFT': post['SQFT'],
            'AGE': post['AGE'],
            'GARAGE': post['GARAGE'],
            'PROPTYPE_CAT': post['PROPTYPE_CAT'],
            'STYLE_CAT': post['STYLE_CAT'],
            'ZIP_CAT': post['ZIP_CAT'],
            'SOLDDATE_CAT': post['SOLDDATE_CAT'],
        })

In [52]:
# build control set
post_flips = pd.DataFrame.from_dict(post_flips)
print(post_flips.shape)

unflipped_homes = controlHomes[['MLSNUM','SOLDPRICE','DOM','BEDS','BATHS','SQFT','AGE','GARAGE',
                                'PROPTYPE_CAT','STYLE_CAT','ZIP_CAT','SOLDDATE_CAT']]
unflipped_homes.insert(0, 'FLIPPABLE', 0)
control_data = pd.concat([unflipped_homes, post_flips], ignore_index=True)

print(control_data.shape)

(3788, 13)
(129598, 13)


# Combining Test and Control Data

Clean and combine the two data frames. Know we will have an unbalanced data set, and will need to account for that in our analysis.

In [54]:
flip_df = pd.DataFrame.from_dict(flip_data)
control_df = pd.DataFrame.from_dict(control_data)

In [55]:
flip_df.head(20)
print(flip_df.shape)
control_df.head(20)
print(control_df.shape)

(3788, 13)
(129598, 13)


In [56]:
# combine test and control
merged_test_and_control_data = pd.concat([flip_df, control_df], ignore_index=True)
print(merged_test_and_control_data.shape)

# remove bad listings for now
bad_data = ['72250832','71902243','72214658','72099376','72032454','72027853','72018311',
            '71955378','72045937','72133139','72144618']
merged_test_and_control_data = merged_test_and_control_data[~merged_test_and_control_data['MLSNUM'].isin(bad_data)]

# clean up data that should be ints or floats
merged_test_and_control_data[['SOLDPRICE', 'AGE']] = merged_test_and_control_data[['SOLDPRICE', 'AGE']].astype(str)

merged_test_and_control_data['SOLDPRICE'] = list(map(lambda x: float(x.replace(",", "")), merged_test_and_control_data['SOLDPRICE']))

merged_test_and_control_data['AGE'] = list(map(lambda x: int(x.replace(",", "")), merged_test_and_control_data['AGE']))
merged_test_and_control_data[['MLSNUM', 'DOM', 'GARAGE','SOLDDATE_CAT']] = merged_test_and_control_data[['MLSNUM', 'DOM', 'GARAGE', 'SOLDDATE_CAT']].astype(int)
merged_test_and_control_data['FLIPPABLE'] = merged_test_and_control_data['FLIPPABLE'].astype(bool)

# verify
merged_test_and_control_data.info()
print(merged_test_and_control_data.shape)
merged_test_and_control_data.head(20)

(133386, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133375 entries, 0 to 133385
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   FLIPPABLE     133375 non-null  bool   
 1   MLSNUM        133375 non-null  int64  
 2   SOLDPRICE     133375 non-null  float64
 3   DOM           133375 non-null  int64  
 4   BEDS          133375 non-null  int64  
 5   BATHS         133375 non-null  float64
 6   SQFT          133375 non-null  int64  
 7   AGE           133375 non-null  int64  
 8   GARAGE        133375 non-null  int64  
 9   PROPTYPE_CAT  133375 non-null  int64  
 10  STYLE_CAT     133375 non-null  int64  
 11  ZIP_CAT       133375 non-null  int64  
 12  SOLDDATE_CAT  133375 non-null  int64  
dtypes: bool(1), float64(2), int64(10)
memory usage: 13.4 MB
(133375, 13)


,FLIPPABLE,MLSNUM,SOLDPRICE,DOM,BEDS,BATHS,SQFT,AGE,GARAGE,PROPTYPE_CAT,STYLE_CAT,ZIP_CAT,SOLDDATE_CAT
0,True,71425748,50000.0,1077,4,2.0,1984,95,0,2,458,168,735998
1,True,71477127,125000.0,724,4,2.0,2333,67,0,0,1,207,736018
2,True,71478495,720500.0,828,4,3.5,4196,20,3,0,3,313,735978
3,True,71485790,160000.0,1140,3,1.5,1350,58,0,0,6,71,736215
4,True,71614526,335000.0,809,3,2.0,1270,38,1,0,6,111,736024
5,True,71633454,364000.0,106,4,1.5,1462,118,0,0,3,10,736009
6,True,71661835,135000.0,760,6,2.0,2315,98,0,2,458,45,735983
7,True,71701065,240000.0,343,4,2.0,1346,38,0,0,1,233,735982
8,True,71716837,440000.0,42,3,3.0,2046,81,1,0,1,44,736039
9,True,71745072,215000.0,1369,3,3.5,2112,31,2,0,3,368,736048


In [57]:
merged_test_and_control_data.to_csv('./outputs/merged_test_and_control_data.csv')

# Profit margins

This will spit out a csv of pre and post values of the FLIPs, with a profit margin:

In [ ]:
# build flip set
flips = [];
for mls in flippedHomes['MLSNUM']:
    homes = getHome(flippedHomes, mls)
    # check if home has 2 mls rows
    if(homes.shape[0] >= 2):
        pre = homes.iloc[0]
        post = homes.iloc[1]
        # check if it's not the same mls number
        if pre['MLSNUM'] == post['MLSNUM']:
            continue
        # find pre and post flip values
        if pd.to_datetime(pre['SOLDDATE']) > pd.to_datetime(post['SOLDDATE']):
            # post < pre: swap them
            swap = pre
            pre = post
            post = swap
        flips.append({
            'PRE-FLIP-MLS': pre['MLSNUM'],
            'POST-FLIP-MLS': post['MLSNUM'],
            'PRE-FLIP-SOLD-DATE': pre['SOLDDATE'],
            'POST-FLIP-SOLD-DATE': post['SOLDDATE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PRE-SOLD-PRICE': pre['SOLDPRICE'],
            'POST-SOLD-PRICE': post['SOLDPRICE'],
            'PROFIT': int(post['SOLDPRICE']) - int(pre['SOLDPRICE'])
        })
        
flips = pd.DataFrame.from_dict(flips)
flips.to_csv('./outputs/flips.csv')
flips.head()